In [1]:
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

모형 평가

오버피팅(overfitting)과 언더피팅(underfitting)  
오버피팅은 특정 데이터셋에 과도하게 적합된 것을 의미한다. 오버피팅이 발생하는 경우, 얼핏 정확도가 높아 보이지만 특정 데이터셋에만 적합되어 <u>알려지지 않은 데이터에 대한 예측력은 낮아</u>지게 된다. 언더피팅은 데이터셋에 적합이 잘되지 않은, 즉 과소적합된 것을 의미한다.

머신러닝을 통해 모형을 학습하는 이유는 데이터의 종류와 상관없이 일반화할 수 있는 모형을 생성하는 것이다. 주어진 데이터셋에 대해 오버피팅이나 언더피팅이 발생한다면 새로운 데이터에 적용할 수 있는 좋은 모형이라고 말하기 어렵다.

오버피팅된 모형을 새로운 데이터셋에 적용한다면 학습 데이터셋과는 큰 오차를 보인다. 반대로 언더피팅은 트레이닝 데이터 셋과 테스트 데이터셋 모두 큰 오차를 보인다.

편향-분산 트레이드오프(bias-variance tradeoff)  
편향 분산 트레이드오프란, <u>편향이 낮을수록 분산은 커지고, 반대로 편향이 높을수록 분산이 작아지는 경향</u>이 있다는 것을 의미한다.

분산이 높은 현상은 주로 복잡한 모형에 나타나고 모형이 복잡하다는 말은 오버피팅이 발생할 가능성이 높다는 뜻이다. 즉, 복잡한 모형일수록 오버피팅이 발생할 가능성이 높으며, 이는 분산이 커진다는 것을 의미한다.  
반대로 편향이 큰 현상은 주로 간단한 모형일 때 나타나는데 모형이 간단하다는 말은 언더피팅이 발생할 가능성이 높다는 뜻이다. 간단한 모형일수록 언더피팅이 발생할 가능성이 높으며, 이에 따라 편향이 커질 수 있다.

크로스 벨리데이션(cross validation, 교차 검증)  
모형을 생성한 후 실제 데이터에 적용해 보고 성능을 평가해야 하는데, 데이터셋 전체를 학습에 사용하면 새롭게 적용할 데이터가 없어서 문제가 발생되기 때문에 전체 데이터를 트레이닝 데이터와 테스트 데이터로 분할해서 사용한다. 트레이닝 데이터는 학습하는 데 사용되고, 테스트 데이터는 학습 시에는 사용되지 않고 모형의 성능을 평가할 때 사용한다.

머신러닝 알고리즘을 적용할 때 다양한 하이퍼파라미터에 대해 여러 가지 모형 후보군을 생성하고 평가한 수 최종 모형을 선택하게 된다. 이때, 파라미터는 모형 내부에서 데이터에 의해 추정되는 값이고, 하이퍼파라미터는 사용자가 직접 정하는 값이다.

하이퍼파라미터를 결정하는 과정에서 트레이닝 데이터와 테스트 데이터만 존재한다면 테스트 데이터에 의해 최종 모형의 파라미터가 결정된다. 즉, 모형의 하이퍼파라미터가 테스트 데이터에 의존한다는 뜻이다.

이 문제를 해결하기 위해 트레이닝 데이터의 일부를 밸리데이션 데이터로 사용한다. 즉, 트레이닝 데이터는 파라미터를 구하는데 사용하고, 베리데이션 데이터는 하이퍼파라미터를 정하는데 사용한다.

주어진 데이터셋에 대해서 트레이닝 데이터, 밸리데이션 데이터, 테스트 데이터로 분할할 수 있는 다양한 조합 방법이 존재한다. 이처럼 다양한 조합을 통해 모형의 성능을 검증하는 것을 크로스-밸리데이션이라고 한다.

파이프라인  
파이썬을 활용한 머신러닝 실습 과정에서 파이프라인을 사용하면 데이터 전처리와 학습 모형을 연결해 코드를 간겨화 할 수 있다.

In [3]:
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [4]:
raw_boston = datasets.load_boston()
X = raw_boston.data
y = raw_boston.target

파이프라인을 사용하지 않은 코드

In [5]:
# 트레이닝/테스트 데이터 분할
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=7)

# 표준화 스케일링
std_scale = StandardScaler()
X_tn_std = std_scale.fit_transform(X_tn)
X_te_std = std_scale.transform(X_te)

# 학습
clf_linear = LinearRegression()
clf_linear.fit(X_tn_std, y_tn)

# 예측
pred_linear = clf_linear.predict(X_te_std)

# 평가
mean_squared_error(y_te, pred_linear)

29.51513779019758

파이프라인을 사용한 코드

In [6]:
# 트레이닝/테스트 데이터 분할
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=7)

# 표준화 스케일링
# std_scale = StandardScaler()
# X_tn_std = std_scale.fit_transform(X_tn)
# X_te_std = std_scale.transform(X_te)
# 학습
# clf_linear = LinearRegression()
# clf_linear.fit(X_tn_std, y_tn)
# 예측
# pred_linear = clf_linear.predict(X_te_std)

# 파이프라인 => 표준화 스케일링 + 학습 객체 생성
linear_pipeline = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('linear_regression', LinearRegression())
    ]
)

# 학습
linear_pipeline.fit(X_tn, y_tn)

# 예측
pred_linear = linear_pipeline.predict(X_te)

# 평가
mean_squared_error(y_te, pred_linear)

29.51513779019758

그리드 서치(grid search)  
그리드 서치는 머신러닝 과정에서 관심있는 매개 변수들을 대상으로 학습 가능하도록 만드는 방식을 의미한다.

K-최근접 이웃 알고리즘 사용시 1부터 10까지의 K값 후보중 가장 높은 성능을 보이는 K값 정하기

In [7]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [8]:
# 붓꽃 데이터 불러오기
raw_iris = datasets.load_iris()

# 붓꽃 데이터에서 피처/타겟 데이터 읽기
X = raw_iris.data
y = raw_iris.target

# 트레이닝/테스트 데이터 분할
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)

# 표준화 스케일링
std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

best_accuracy = 0 # 최고 정확도를 기억할 변수를 선언하고 0으로 초기화한다.
# 1부터 10까지의 K값 후보중 가장 높은 성능을 보이는 K값을 정한다.
for k in range(1, 11):
    clf_knn = KNeighborsClassifier(n_neighbors=k) # kNN 객체를 생성한다.
    clf_knn.fit(X_tn_std, y_tn) # 트레이닝 데이터로 학습한다.
    knn_pred = clf_knn.predict(X_te_std) # 테스트 데이터로 예측한다.
    accuracy = accuracy_score(y_te, knn_pred) # 테스트 데이터 실제값과 테스트 데이터 예측값으로 정확도를 계산한다.
    # 가장 정확도가 높은 k를 계산한다.
    print('k: {:2d}, best_accuracy: {:6.3f}, accuracy: {:6.3f}'.format(k, best_accuracy, accuracy))
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        final_k = k
    # ===== if
# ===== for
print('=' * 80)
print('k: {:2d}, accuracy: {:.3%}'.format(final_k, best_accuracy))

k:  1, best_accuracy:  0.000, accuracy:  0.921
k:  2, best_accuracy:  0.921, accuracy:  0.947
k:  3, best_accuracy:  0.947, accuracy:  0.974
k:  4, best_accuracy:  0.974, accuracy:  0.974
k:  5, best_accuracy:  0.974, accuracy:  0.974
k:  6, best_accuracy:  0.974, accuracy:  0.974
k:  7, best_accuracy:  0.974, accuracy:  0.974
k:  8, best_accuracy:  0.974, accuracy:  0.974
k:  9, best_accuracy:  0.974, accuracy:  0.974
k: 10, best_accuracy:  0.974, accuracy:  0.974
k:  3, accuracy: 97.368%


손실 함수(loss function)와 비용 함수(cost function)의 개념

손실 함수는 머신러닝을 통해 생성한 모형이 실제값과 얼마나 차이가 나는지 즉, 손실 정도를 수치로 나타내는 함수로 모형 손실은 예측값과 실제값의 차이를 이용해 측정한다.

손실 함수와 비슷하게 비용 함수라는 개념도 존재하는데 손실 함수는 각 데이터 포인트에 대해 예측값과 실제값의 차이를 나타내지만, 비용 함수는 데이터셋 전체를 대상으로 하는 손실을 의미한다. 엄밀하게 말하면 서로 다르다고 할 수도 있으나 실제로는 손실 함수와 비용 함수를 구분없이 사용하기도 한다.

L1 손실 함수

손실 함수에는 크게 L1 손실(L1 loss)과 L2 손실(L2 loss)이 존재한다. L1 손실은 다른말로 L1 비용(L1 cost)이라고도 부르며, 아래와 같이 표현한다.

$$L1\,Loss = \sum |y_{true} - y_{predict}|$$

$y_{true}$는 실제값을 의미하고, $y_{predict}$는 학습 모형을 이용해 예측한 값을 의미한다.

즉, L1 손실은 실제값과 예측값의 차이에 절대값을 취한 것으로 실제값과 예측값의 차이를 줄이는 것이 학습 목적이다.  
L1 손실과 관련된 비용 함수로 MAE(Mean Absolute Error)가 있다.

$$MAE = \frac{1}{n} \sum_{i=1}^{n}{|y_i - \hat y|}$$

L2 손실 함수

L2 손실은 실제값과 예측값의 차이에 제곱을 취함으로써 구할 수 있다.

$$L2\,Loss = \sum (y_{true} - y_{predict})^2$$

L2 손실을 이용한 비용 함수에는 MSE(Mean Squared Error), RMSE(Root Mean Squared Error)가 존재한다.

$$MSE = \frac{1}{n} \sum_{i=1}^{n}{(y_i - \hat y)^2}, \; RMSE = \sqrt{MSE}$$

MSE는 흔히 사용하는 비용 함수로 실제값과 예측값의 차이의 평균을 의미한다. RMSE는 MSE에 제곱근을 취한 형태이다. MSE를 구하는 과정에서 실제값과 예측값을 제곱하므로 MSE는 이상치(outlier)의 변화에 민감하다. 반면에 MAE나 RMSE는 이상치와 상관없이 안정된 값을 보여준다. 머신러닝 모형에서 이상치에 중점을 두고싶다면 MSE를 사용하고, 그렇치 않다면 MAE나 RMSE를 사용할 수 있다.

엔트로피(entropy)

엔트로피는 정보 이론에서 사용하는 개념으로 확률 변수의 불확실성 정도를 측정하기 위해 사용한다.

■ 엔트로피

$$Entropy(P) = - \sum P(x)logP(x) = -E(logP(x))$$

위 엔트로피 식은 $Entropy(P)$로 표기했지만 $H(P)$ 혹은 $H(X)$라고 쓰기도 하며 엔트로피는 의사결정 트리에서 주로 사용한다.

■ 크로스 엔트로피

$$CrossEntropy(P, Q) = - \sum P(x)logQ(x) = -E_p(logQ(x))$$

위 식은 크로스 엔트로피라고 하는데, 엔트로피는 하나의 분포를 대상으로 하는 반면, 크로스 엔트로피는 두 분포 $P(x)$, $Q(x)$를 대상으로 엔트로피를 측정해 두 분포간의 차이를 계산한다. 머신러닝에서 크로스 엔트로피를 사용할 대는 $P(x)$를 실제 모형의 분포, $Q(x)$ 추정 모형의 분포라고 설정한다.

■ 쿨백-라이블러 발산(Kullback-Leibler Divergence)

$$
\begin{aligned}
D_{KL}(P||Q) =& \sum P(x)log \frac{P(x)}{Q(x)} \\
=& -\sum P(x)logQ(x) + \sum P(x)logP(x) \\
=& -E_p(log \frac{P(x)}{Q(x)})
\end{aligned}
$$

위 식은 쿨백-라이블러 발산(KLD)라는 개념이다. 크로스 엔트로피와 KLD는 머신러닝에서 자주 사용되는 손실 함수이다. KLD는 다른 말로 상대적 엔트로피(relative entropy)라고도 부른다.

모형 성능 평가

모형 성능 평가에 필요한 개념  
정답으로 분류되는 경우  
데이터를 양성(positive)으로 예측했을 때, 실제값도 양성일 때 => True Positive(TP)  
데이터를 음성(negative)으로 예측했을 때, 실제값도 음성일 때 => True Negative(TN)  
오답으로 분류되는 경우  
데이터를 양성(positive)으로 예측했을 때, 실제값이 음성일 때 => False Positive(FP)  
데이터를 음성(negative)으로 예측했을 때, 실제값이 양성일 때 => False Negative(FN)  

■ 정밀도(Precision): 양성으로 예측했을 때, 실제로 양성으로 나타나는 비율 => $\frac{TP}{TP + FP}$  
■ 재현율(Recall): 실제 양성에 해당하는 데이터가 양성으로 나타나는 비율 => $\frac{TP}{TP + FN}$  
■ 특이도(Specificity): 음성으로 예측했을 때, 실제로 음성으로 나타나는 비율 => $\frac{TN}{FP + TN}$  
■ False Positive Rate(FPR): 실제 음성에 해당하는 데이터가 양성으로 나타나는 비율 => $\frac{FP}{FP + TN}$  
■ 정확도(Accuracy): 전체 데이터 중 양성으로 분류되는 비율 => $\frac{TP + TN}{TP + TN + FP + FN}$  
■ 에러율(Error Rate): 전체 데이터 중 음성으로 분류되는 비율 => $\frac{FP + FN}{TP + TN + FP + FN}$  
■ Receiver Operating Characteristic(ROC) 커브: X축에 FPR을 놓고 Y축에 재현율의 값을 비교한다.

머신러닝에서 우리가 풀어야 할 문제의 종류에 따라 성능 평가 방법이 달라지기도 한다.  

분류 문제에서의 성능 평가  